In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
# d = [[1, 2], [1, 2]]
# n = 2   # 两个设施点
# m = 2   # 两个需求点
d = [[1, 2], [3, 4], [5, 6]]
n = 2   # 两个设施点
m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j - 1] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
def gnrt_Hd(n):
  Hd = np.zeros((2**n, 2**n)).astype(np.complex128)
  for i in range(n):
    j = (i + 1) % n
    Hd += (add_in_target(n, i, GateX) @ add_in_target(n, j, GateX) + add_in_target(n, i, GateY) @ add_in_target(n, j, GateY))
  return -Hd


In [7]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    for i in range(m):
      qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n)), range(n * (m + i) , n * (m + i + 1)))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [8]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌───────────┐┌─────────┐ ┌───────────┐┌─────────┐ ┌───────────┐»
    q_0: ┤ H ├┤0          ├┤ Rx(π/4) ├─┤0          ├┤ Rx(π/4) ├─┤0          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_1: ┤ H ├┤1          ├┤ Rx(π/4) ├─┤1          ├┤ Rx(π/4) ├─┤1          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_2: ┤ H ├┤2          ├┤ Rx(π/4) ├─┤2          ├┤ Rx(π/4) ├─┤2          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_3: ┤ H ├┤3          ├┤ Rx(π/4) ├─┤3          ├┤ Rx(π/4) ├─┤3          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_4: ┤ H ├┤4          ├┤ Rx(π/4) ├─┤4          ├┤ Rx(π/4) ├─┤4          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_5: ┤ H ├┤5          ├┤ Rx(π/4) ├─┤5          ├┤ Rx(π/4) ├─┤5          ├»
         ├───┤│           │├─────────┴┐│           │

In [9]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [10]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [11]:
test(40, 1, np.full(1 * 2, np.pi))

Iteration 10, Result: 271.3645
Iteration 20, Result: 153.3285
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 153.644
       x: [ 4.187e+00  4.636e+00]
    nfev: 29
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
11101010000000 1.5% 132
11101010000010 1.2% 172
11101010001000 1.2% 172
11101010100000 1.1% 172
11101010101000 0.9% 212
11101010100010 0.9% 212
11101010001010 0.9% 212
11101010000001 0.8% 92
11101010000100 0.8% 92
11101010010000 0.8% 92
11101010101010 0.7% 252
11101010100001 0.6% 132
11101010011000 0.6% 132
11101010001100 0.6% 132
11101010001001 0.6% 132
11101010100100 0.6% 132
11101010000110 0.6% 132
11101010010010 0.6% 132
11101010110000 0.6% 132
11101010000011 0.6% 132


In [12]:
test(40, 2, np.full(2 * 2, np.pi))

Iteration 10, Result: 232.648
Iteration 20, Result: 354.3375
Iteration 30, Result: 148.9185
Iteration 40, Result: 140.467
Iteration 50, Result: 128.338
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 126.3585
       x: [ 4.032e+00  3.604e+00  3.938e+00  3.137e+00]
    nfev: 55
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000000 29.0% 11
00101010000000 10.8% 129
11101010000000 4.5% 132
00101010100000 3.6% 249
00101010000010 3.6% 249
00101010001000 3.6% 249
11101010010000 2.4% 92
11101010000100 2.3% 92
11101010000001 2.3% 92
10101010000100 1.9% 51
10101010000001 1.9% 51
10101010010000 1.8% 51
01101010000000 1.6% 250
10011010000000 0.8% 92
10100110000000 0.8% 92
10101001000000 0.8% 92
01101010000100 0.8% 210
00101010101000 0.8% 369
01101010010000 0.8% 210
01101010000001 0.8% 210


In [13]:
test(40, 3, np.full(3 * 2, np.pi))

Iteration 10, Result: 262.2695
Iteration 20, Result: 137.5185
Iteration 30, Result: 179.706
Iteration 40, Result: 164.0395
Iteration 50, Result: 134.8735
Iteration 60, Result: 135.5105
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 135.5105
       x: [ 4.143e+00  3.134e+00  4.178e+00  3.132e+00  4.192e+00
            3.139e+00]
    nfev: 61
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
01010101000000 3.0% 13
11101010000000 1.9% 132
11101001000010 1.9% 93
11100110001000 1.9% 93
11011010100000 1.9% 93
11011001100010 1.1% 54
11010110101000 1.1% 54
11100101001010 1.1% 54
10101010000000 1.0% 11
01101010000000 0.9% 250
11101010010000 0.8% 92
11101010000001 0.8% 92
11101010000100 0.8% 92
11010101101010 0.7% 15
01010101001000 0.7% 53
01010101100000 0.7% 53
01010101000010 0.6% 53
11100110001001 0.6% 53
11011010100001 0.6% 53
11100110011000 0.6

In [14]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 241.3135
Iteration 20, Result: 329.751
Iteration 30, Result: 217.4025
Iteration 40, Result: 220.792
Iteration 50, Result: 250.997
Iteration 60, Result: 201.5625
Iteration 70, Result: 190.4845
Iteration 80, Result: 188.797
Iteration 90, Result: 191.26
Iteration 100, Result: 188.735
Iteration 110, Result: 188.7635
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 189.268
       x: [ 1.779e+00  7.501e-01  1.701e+00  7.641e-01  1.716e+00
            1.796e+00  8.106e-01  1.812e+00  7.476e-01  7.486e-01]
    nfev: 116
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
00010101000000 5.4% 132
01100101000000 2.5% 92
01010110000000 2.5% 92
01011001000000 2.5% 92
10101010000000 1.7% 11
10010101000000 1.1% 254
11011001001000 1.1% 174
11010110000010 1.0% 174
11100101100000 1.0% 174
00010101100000 1.0% 172
00010101001000 0.9% 172
00